# **SignSegmenter: Precision Hand Sign Segmentation with YOLOv5**

This Jupyter Notebook showcases the application of YOLOv5, for instance segmentation tasks on a sign recognition dataset. Our objective is to not only detect but also precisely segment various signs from the dataset, which is essential for applications requiring detailed spatial understanding of the signs' presence in images. By leveraging YOLOv5's powerful detection capabilities and adapting it for instance segmentation, we aim to capture the fine-grained details necessary for accurate sign recognition. This approach involves the innovative use of YOLOv5 for feature extraction, combined with segmentation techniques to delineate the boundaries of each detected sign. The notebook covers the entire workflow, including dataset preparation, model adaptation for instance segmentation, training with transfer learning, and comprehensive evaluation of the segmentation performance.

## **Saving the Current Working Directory**

To facilitate navigation and file management throughout our project on instance segmentation using YOLOv5, it's practical to save the current working directory path. This step ensures that we can easily return to or reference our project's base directory, regardless of any subsequent changes to the working directory during dataset processing, model training, or evaluation stages.

In [2]:
# save the working directory path for later use
HOME = os.getcwd()
print(HOME)

/content


## **Cloning YOLOv5 Repository and Setting Up the Environment**

To leverage YOLOv5 for our instance segmentation project on a sign recognition dataset, we first need to clone the YOLOv5 GitHub repository. This repository contains the necessary codebase, including the model architecture, pre-trained weights, and utility scripts essential for training and inference. Following the cloning process, we navigate into the YOLOv5 directory, which now resides within our project's home directory, to proceed with environment setup and dependency installation.

In [ ]:
# clone project repository and install dependencies
!git clone https://github.com/ultralytics/yolov5.git # clone
%cd {HOME}/yolov5

## **Installing Dependencies and Initializing the Notebook Environment**

After setting up the YOLOv5 repository in our workspace, the next step involves installing the required dependencies and performing initial checks to ensure the environment is correctly configured for our instance segmentation project. This includes installing packages listed in the requirements.txt file, which contains all the necessary Python libraries for YOLOv5 to function properly. Additionally, we'll initialize the notebook environment using a utility function provided by YOLOv5, which performs various checks and setups.

In [5]:
import os

import torch

import utils
from utils.downloads import attempt_download

%pip install -r requirements.txt # install
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-283-g875d9278 Python-3.10.12 torch-2.1.0+cu121 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.4/225.8 GB disk)


## **Downloading Pretrained Weights for Segmentation Models**

To harness the full power of YOLOv5 for our instance segmentation project, we utilize pretrained weights. These weights have been trained on a large dataset, enabling the model to effectively recognize and segment a wide range of objects. Specifically, for segmentation tasks, we focus on downloading weights for models that are tailored for segmentation purposes.

In [ ]:
# download pretrained weights
p5 = ['n', 's', 'm', 'l', 'x']  # P5 models
cls = [f'{x}-seg' for x in p5]  # segmentation models

for x in cls:
    attempt_download(f'{HOME}/yolov5/weights/yolov5{x}.pt')

## **Setting Up Dataset with Roboflow**

We utilize Roboflow, a platform that simplifies the process of managing, preprocessing, and augmenting datasets for machine learning applications. By integrating Roboflow, we can easily access a structured and optimized version of our sign recognition dataset, specifically formatted for use with YOLOv5.

First, we navigate back to the YOLOv5 directory within our project's home directory. We then install the Roboflow library, which provides the necessary tools to interact with the Roboflow platform directly from our notebook. Using the provided API key, we access our specific project on Roboflow and download the sign recognition dataset. This dataset has been preprocessed and arranged to be compatible with YOLOv5's expected input format, streamlining the process of loading data for training and inference.



In [ ]:
%cd {HOME}/yolov5

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="YOUR API KEY")
project = rf.workspace("ss-hwnzd").project("sign_recognition")
dataset = project.version(2).download("yolov5")


## **Reviewing Dataset Configuration**

After downloading the sign recognition dataset through Roboflow, it's important to examine the data.yaml file that accompanies the dataset.

In [ ]:
%cat {dataset.location}/data.yaml

## **Initiating Training for Instance Segmentation**

Now that the dataset is ready and the environment is set up, the next step is to start the training process for instance segmentation on the sign recognition dataset using YOLOv5. This involves using a customized training script segment/train.py, which is adapted for segmentation tasks, along with specifying relevant parameters to tailor the training process to our dataset and objectives.

In [ ]:
%cd {HOME}/yolov5
!python segment/train.py --img 320 --batch 128 --epochs 100 --data {dataset.location}/data.yaml --weights yolov5s-seg.pt --name custom-dataset


## **Displaying Training Results for Instance Segmentation**

After completing the training of your instance segmentation model on the sign recognition dataset with YOLOv5, it's important to evaluate the training performance by examining the results graph. This graph provides a visual representation of the training and validation metrics over time, such as loss and possibly other metrics depending on your segmentation task and the modifications made to the YOLOv5 training script for segmentation.

In [ ]:
display.Image(filename=f'{HOME}/yolov5/runs/train-seg/custom-dataset/results.png', width=1200)

## **Running Validation for Instance Segmentation**

After training your instance segmentation model on the sign recognition dataset, the next crucial step is to validate the model's performance on a separate set of data not seen during training. This validation process assesses the model's generalization capabilities and ensures that it can effectively segment signs in new, unseen images.

In [ ]:
%cd {HOME}/yolov5
!python segment/val.py --weights {HOME}/yolov5/runs/train-seg/custom-dataset/weights/best.pt --data {dataset.location}/data.yaml --img 320  --name custom-dataset


## **Performing Instance Segmentation Prediction on an Example Image**

After validating the instance segmentation model trained on the sign recognition dataset, you might want to see how the model performs on individual images. This step involves running a prediction on an example image from the test or validation set and visually inspecting the model's ability to segment signs accurately.

In [ ]:
if os.path.exists(os.path.join(dataset.location, "test")):
    split_path = os.path.join(dataset.location, "test", "images")
else:
    split_path = os.path.join(dataset.location, "valid", "images")
example_image_name = os.listdir(split_path)[0]
example_image_path = os.path.join(split_path, example_image_name)

%cd {HOME}/yolov5
!python segment/predict.py --img 320 --weights {HOME}/yolov5/runs/train-seg/custom-dataset/weights/best.pt --source {example_image_path} --name custom-dataset
display.Image(filename=f"{HOME}/yolov5/runs/predict-seg/custom-dataset/{example_image_name}", width=600)